In [82]:
from web3 import Web3
from web3.gas_strategies.rpc import rpc_gas_price_strategy
from uniswap import Uniswap
import os
import json
import time
import numpy as np

In [83]:
class TokenTrader:
    def __init__(self, rpc_url, wallet_address, private_key, token_address, 
                 usdt_address, eth_address, price_thresholds, sale_percentages, 
                 min_sell_amount_tokens, etherscan_api_key):
        self.rpc_url = rpc_url
        self.wallet_address = wallet_address
        self.private_key = private_key
        self.token_address = token_address
        self.usdt_address = usdt_address
        self.eth_address = eth_address
        self.price_thresholds = price_thresholds
        self.sale_percentages = sale_percentages
        self.min_sell_amount_tokens = min_sell_amount_tokens
        self.etherscan_api_key = etherscan_api_key
        
        self.web3 = Web3(Web3.HTTPProvider(self.rpc_url))
        self.web3.eth.set_gas_price_strategy(rpc_gas_price_strategy)
        self.uniswap = Uniswap(address=self.wallet_address, private_key=self.private_key, version=2, web3=self.web3, provider=self.rpc_url)
        
        self.eth_decimals = self.get_token_decimals(self.eth_address)
        self.usdt_decimals =self.get_token_decimals(self.usdt_address)
        self.token_decimals = self.get_token_decimals(self.token_address)
        
        self.min_sell_amount = int(self.min_sell_amount_tokens * 10**self.token_decimals)

        self.state_file = 'trader_state.txt'
        self.sold_tokens = self.load_state()


    def get_token_decimals(self, token_address):
        token_contract = self.uniswap.get_token(token_address)
        return token_contract.decimals
        
    def load_state(self):
        if os.path.exists(self.state_file):
            with open(self.state_file, 'r') as file:
                state = json.load(file)
                sold_tokens = [int(token) for token in state['sold_tokens']]
        else:
            sold_tokens = np.zeros_like(self.price_thresholds)
        return sold_tokens

    def save_state(self):
        state = {
            'sold_tokens': [str(token) for token in self.sold_tokens] 
        }
        with open(self.state_file, 'w') as file:
            json.dump(state, file)

    def get_token_balance(self, token_address):
        return self.uniswap.get_token_balance(token_address)
    
    def get_price_from_pool(self, input_token, output_token, decimals):
        try:
            price = self.uniswap.get_price_input(input_token, output_token, 10**decimals)
            return price
        except Exception as e:
            print(f"Ha ocurrido un error al chequear el precio: {e}")
            return None

    def get_token_price_in_usdt(self):
        token_price_in_eth = self.get_price_from_pool(self.token_address, self.eth_address, self.token_decimals)
        eth_price_in_usdt = self.get_price_from_pool(self.eth_address, self.usdt_address, self.eth_decimals)
        if token_price_in_eth is not None and eth_price_in_usdt is not None:
            return (token_price_in_eth / 10**self.eth_decimals) * (eth_price_in_usdt / 10**self.usdt_decimals)
        return None
    
    def calculate_sale_amount(self, price_in_usdt):
        total_to_sell = 0
        lots_to_sell = []
        # Comprobar el precio y calcular la venta potencial por cada umbral
        for i in range(len(self.price_thresholds)):
            if price_in_usdt >= self.price_thresholds[i]:
                max_tokens_at_this_level = self.total_tokens * self.sale_percentages[i]
                if self.sold_tokens[i] < max_tokens_at_this_level:
                    total_to_sell += max_tokens_at_this_level - self.sold_tokens[i]
                lots_to_sell.append((i, total_to_sell))
        return total_to_sell, lots_to_sell
    
    def sell_tokens(self, total_to_sell, price_in_usdt, lots_to_sell):
        print(f"Vendiendo {(total_to_sell / 10**self.token_decimals)} tokens a {price_in_usdt} USDT")
        tx = self.uniswap.make_trade(self.token_address, self.eth_address, int(total_to_sell))
        print(f"Esperando por confirmación de la transacción {tx.hex()}")
        self.web3.eth.wait_for_transaction_receipt(tx)
        print(f"Transacción confirmada: {tx.hex()}")
        for lot_index, amount in lots_to_sell:
            self.sold_tokens[lot_index] += amount
        self.save_state()
    
    def manage_sales(self, balance, price_in_usdt):
        pass
    
    def trade(self):
        while True:
            try:
                balance = int(self.get_token_balance(self.token_address))
                print(f"Balance actual: {(balance / 10**self.token_decimals)} tokens")
                price_in_usdt = self.get_token_price_in_usdt()
                print(f"Precio actual: {price_in_usdt} USDT")
                if price_in_usdt is not None and balance > self.min_sell_amount:
                    self.manage_sales(balance, price_in_usdt)
                else:
                    time.sleep(5)
                    continue
                # Espero 1 segundo
                time.sleep(1)
            except Exception as e:
                print(f"Ha ocurrido un error: {e}")
                time.sleep(1)